In [1]:
%pip install gurobipy
%pip install cvxpy
import gurobipy
import cvxpy as cp
import numpy as np
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
rng = np.random.default_rng()

subdistricts = range(80)
#["Alpha", "Beta", "Gamma", "Delta", "Epsilon", "Zeta", "Eta", "Theta", "Iota", "Kappa", "Mu"]
timeslots = ["MA","MP", "WA","WP", "FA","FP", "TA","TP", "HA","HP", "SA","SP"]
timecombos = []
trucks = 50

for a in ["MA","MP"]:
    for b in ["WA","WP"]:
        for c in ["FA","FP"]:
            timecombos.append((a,b,c))
for a in ["TA","TP"]:
    for b in ["HA","HP"]:
        for c in ["SA","SP"]:
            timecombos.append((a,b,c))
print(timecombos)

sub_ct = len(subdistricts)
ts_ct = len(timeslots)
com_ct = len(timecombos)

[('MA', 'WA', 'FA'), ('MA', 'WA', 'FP'), ('MA', 'WP', 'FA'), ('MA', 'WP', 'FP'), ('MP', 'WA', 'FA'), ('MP', 'WA', 'FP'), ('MP', 'WP', 'FA'), ('MP', 'WP', 'FP'), ('TA', 'HA', 'SA'), ('TA', 'HA', 'SP'), ('TA', 'HP', 'SA'), ('TA', 'HP', 'SP'), ('TP', 'HA', 'SA'), ('TP', 'HA', 'SP'), ('TP', 'HP', 'SA'), ('TP', 'HP', 'SP')]


In [3]:
#randomly generates time cost and negative impact per time slot per subdistrict

tc_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    for j in range(ts_ct):
        tc_arr[i][j] = max(0.5, rng.normal(2.0, 0.5))

ni_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    for j in range(0, ts_ct, 2):
        ni_arr[i][j] = max(0.2, rng.normal(3.0, 1))
        ni_arr[i][j + 1] = 1.5 * ni_arr[i][j]

In [4]:
#randomly generates time cost and negative impact per time slot per subdistrict
#assuming in one district, weekday AM and nonfri/sat PM have same time cost
tc_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    ams = max(0.5, rng.normal(2.0, 0.5))
    pms = max(0.5, rng.normal(2.0, 0.5))
    for j in [0, 2, 4, 6, 8]:
        tc_arr[i][j] = ams
    for j in [1, 3, 7, 9]:
        tc_arr[i][j] = pms
    for j in [5, 10, 11]:
        tc_arr[i][j] = max(0.5, rng.normal(2.0, 0.5))

ni_arr = np.zeros((sub_ct, ts_ct))
for i in range(sub_ct):
    for j in range(0, ts_ct, 2):
        ni_arr[i][j] = max(0.2, rng.normal(3.0, 1))
        ni_arr[i][j + 1] = 1.5 * ni_arr[i][j]

print(tc_arr)

[[1.96963699 2.5897122  1.96963699 2.5897122  1.96963699 2.51614848
  1.96963699 2.5897122  1.96963699 2.5897122  2.16914319 1.754978  ]
 [2.19233927 2.10416434 2.19233927 2.10416434 2.19233927 2.50037183
  2.19233927 2.10416434 2.19233927 2.10416434 2.54447104 1.21328129]
 [2.0522542  2.45863564 2.0522542  2.45863564 2.0522542  1.94073967
  2.0522542  2.45863564 2.0522542  2.45863564 1.64110369 1.47397371]
 [2.31821643 1.54772326 2.31821643 1.54772326 2.31821643 1.60876735
  2.31821643 1.54772326 2.31821643 1.54772326 1.86671869 1.37204898]
 [2.45759992 1.94690981 2.45759992 1.94690981 2.45759992 1.59117155
  2.45759992 1.94690981 2.45759992 1.94690981 1.97290683 2.2396837 ]
 [1.93319064 1.71039291 1.93319064 1.71039291 1.93319064 2.05453025
  1.93319064 1.71039291 1.93319064 1.71039291 2.12162239 2.59573381]
 [2.15503482 3.22551129 2.15503482 3.22551129 2.15503482 2.11233045
  2.15503482 3.22551129 2.15503482 3.22551129 1.94520693 2.4746022 ]
 [3.29503253 1.82107738 3.29503253 1.8210

In [5]:
# Array representing time costs for subdistrict i with collection at time slot j
time_costs_slot = {}
for i in range(sub_ct):
    for j in range(ts_ct):
        time_costs_slot[i, timeslots[j]] = tc_arr[i][j]
# populate somehow

# Array representing negative impact for subdistrict i with collection at time slot j
negative_impact_slot = {}
for i in range(sub_ct):
    for j in range(ts_ct):
        negative_impact_slot[i, timeslots[j]] = ni_arr[i][j]
# populate somehow

for key, value in negative_impact_slot.items():
    print(key, value)

# Array representing negative impact for subdistrict i with time combo j
negative_impact_combo = np.zeros((sub_ct, 16))
for i in range(sub_ct):
    for j in range(len(timecombos)):
        total = 0
        for time_slot in timeslots:
            total += (time_slot in timecombos[j]) * negative_impact_slot[i, time_slot]
        negative_impact_combo[i][j] = total

print(negative_impact_combo)

(0, 'MA') 4.504641414948576
(0, 'MP') 6.756962122422864
(0, 'WA') 2.5843767044524615
(0, 'WP') 3.8765650566786922
(0, 'FA') 3.1399560801153323
(0, 'FP') 4.709934120172998
(0, 'TA') 2.49819463709056
(0, 'TP') 3.7472919556358404
(0, 'HA') 4.465744165567242
(0, 'HP') 6.698616248350863
(0, 'SA') 2.3428371248885265
(0, 'SP') 3.5142556873327897
(1, 'MA') 3.4291651768200815
(1, 'MP') 5.143747765230122
(1, 'WA') 1.5252851647601797
(1, 'WP') 2.2879277471402695
(1, 'FA') 3.334404692678854
(1, 'FP') 5.001607039018281
(1, 'TA') 3.092020432919718
(1, 'TP') 4.638030649379577
(1, 'HA') 3.309543353213708
(1, 'HP') 4.964315029820562
(1, 'SA') 3.476886347618693
(1, 'SP') 5.215329521428039
(2, 'MA') 2.6729993965247214
(2, 'MP') 4.009499094787082
(2, 'WA') 2.4408954073786315
(2, 'WP') 3.6613431110679473
(2, 'FA') 3.46486870741102
(2, 'FP') 5.19730306111653
(2, 'TA') 2.9306558885948037
(2, 'TP') 4.395983832892206
(2, 'HA') 2.4554186587975364
(2, 'HP') 3.6831279881963046
(2, 'SA') 2.4794746738095537
(2, 'SP

In [6]:
avs = {}
for subdistrict in subdistricts:
    for time_combo in timecombos:
        avs[subdistrict, time_combo] = cp.Variable(boolean = True)

# Define objective
neg_imp_total = sum(negative_impact_combo[i][j] * avs[subdistricts[i],timecombos[j]] for i in range(sub_ct) for j in range(com_ct))

# Create constraints
constraints = []
for timeslot in timeslots:
    constraints += [ sum(time_costs_slot[i, timeslot] * (timeslot in timecombos[j]) * avs[subdistricts[i], timecombos[j]] for i in range(sub_ct) for j in range(com_ct)) <= trucks]
for subdistrict in subdistricts:
    constraints += [ sum(avs[subdistrict, time_combo] for time_combo in timecombos) == 1]

  # Solve problem
prob = cp.Problem(cp.Minimize(neg_imp_total), constraints)
prob.solve(solver = cp.GUROBI)

Restricted license - for non-production use only - expires 2025-11-24


706.6757855529188

In [7]:
print("objective =", prob.value)

assignments = {}
for subdistrict in subdistricts:
    for time_combo in timecombos:
        if avs[subdistrict, time_combo].value == 1:
            assignments[subdistrict] = [time_combo]
            
for key, value in assignments.items():
    print(key, value)

objective = 706.6757855529188
0 [('TA', 'HA', 'SP')]
1 [('MA', 'WP', 'FA')]
2 [('TA', 'HA', 'SA')]
3 [('TA', 'HA', 'SP')]
4 [('TP', 'HA', 'SA')]
5 [('TP', 'HP', 'SP')]
6 [('MA', 'WP', 'FA')]
7 [('MP', 'WP', 'FA')]
8 [('TP', 'HA', 'SA')]
9 [('TA', 'HP', 'SA')]
10 [('MA', 'WA', 'FA')]
11 [('TA', 'HA', 'SA')]
12 [('MP', 'WA', 'FA')]
13 [('MA', 'WA', 'FA')]
14 [('TA', 'HA', 'SA')]
15 [('MA', 'WA', 'FA')]
16 [('MA', 'WP', 'FA')]
17 [('TA', 'HA', 'SA')]
18 [('TA', 'HA', 'SA')]
19 [('MA', 'WA', 'FP')]
20 [('TP', 'HP', 'SA')]
21 [('TA', 'HA', 'SA')]
22 [('TA', 'HA', 'SP')]
23 [('MA', 'WP', 'FA')]
24 [('MA', 'WA', 'FA')]
25 [('TA', 'HP', 'SA')]
26 [('MA', 'WA', 'FA')]
27 [('MA', 'WA', 'FA')]
28 [('MA', 'WA', 'FP')]
29 [('TA', 'HA', 'SA')]
30 [('TA', 'HA', 'SA')]
31 [('TA', 'HA', 'SA')]
32 [('MP', 'WA', 'FA')]
33 [('TA', 'HA', 'SP')]
34 [('MP', 'WP', 'FP')]
35 [('TP', 'HP', 'SA')]
36 [('MA', 'WA', 'FA')]
37 [('MA', 'WA', 'FA')]
38 [('TA', 'HP', 'SA')]
39 [('MA', 'WA', 'FA')]
40 [('TA', 'HA', 'SP